In [1]:
import pandas as pd
import re
import json
import ast
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from Levenshtein import distance as lev
import time


In [2]:
food_path = "1M_recipes_with_nutritional_info.json"
flavor_path = "flavor_DB.json"

In [3]:
def flatten(path, cols, meta):
   # load data using Python JSON module
    with open(path,'r') as f:
        data = json.loads(f.read())
        
    # Flatten data
    df = pd.json_normalize(
        data, 
        record_path =cols, 
        meta=meta
    )
    return df


In [4]:
food_df = pd.read_json(food_path)
food_df.info()
food_df.iloc[0:5, :]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51235 entries, 0 to 51234
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   fsa_lights_per100g   51235 non-null  object
 1   id                   51235 non-null  object
 2   ingredients          51235 non-null  object
 3   instructions         51235 non-null  object
 4   nutr_per_ingredient  51235 non-null  object
 5   nutr_values_per100g  51235 non-null  object
 6   partition            51235 non-null  object
 7   quantity             51235 non-null  object
 8   title                51235 non-null  object
 9   unit                 51235 non-null  object
 10  url                  51235 non-null  object
 11  weight_per_ingr      51235 non-null  object
dtypes: object(12)
memory usage: 4.7+ MB


,fsa_lights_per100g,id,ingredients,instructions,nutr_per_ingredient,nutr_values_per100g,partition,quantity,title,unit,url,weight_per_ingr
0,"{'fat': 'green', 'salt': 'green', 'saturates':...",000095fc1d,"[{'text': 'yogurt, greek, plain, nonfat'}, {'t...",[{'text': 'Layer all ingredients in a serving ...,"[{'fat': 0.8845044000000001, 'nrg': 133.80964,...","{'energy': 81.12946131894766, 'fat': 2.1401392...",train,"[{'text': '8'}, {'text': '1'}, {'text': '1/4'}]",Yogurt Parfaits,"[{'text': 'ounce'}, {'text': 'cup'}, {'text': ...",http://tastykitchen.com/recipes/breakfastbrunc...,"[226.796, 152.0, 30.5]"
1,"{'fat': 'red', 'salt': 'orange', 'saturates': ...",00051d5b9d,"[{'text': 'sugars, granulated'}, {'text': 'oil...",[{'text': 'Cream sugar and butter together til...,"[{'fat': 0.0, 'nrg': 384.0, 'pro': 0.0, 'sat':...","{'energy': 477.09640393594606, 'fat': 23.41248...",val,"[{'text': '1/2'}, {'text': '3/4'}, {'text': '1...","Salt Free, Low Cholesterol Sugar Cookies Recipe","[{'text': 'cup'}, {'text': 'cup'}, {'text': 'c...",http://cookeatshare.com/recipes/salt-free-low-...,"[100.80000000000001, 168.0, 21.25, 5.166666666..."
2,"{'fat': 'orange', 'salt': 'orange', 'saturates...",00059b093b,"[{'text': 'chicken, broilers or fryers, wing, ...","[{'text': 'Preheat oven to 400 degrees F.'}, {...","[{'fat': 174.859716, 'nrg': 2599.0821599999995...","{'energy': 208.05898280960727, 'fat': 14.29704...",train,"[{'text': '3'}, {'text': '1'}, {'text': '1/2'}...",Honey Sriracha Chicken Wings,"[{'text': 'pound'}, {'text': 'teaspoon'}, {'te...",http://tastykitchen.com/recipes/main-courses/h...,"[1360.7759999999998, 6.0, 1.15, 13.5, 42.59999..."
3,"{'fat': 'orange', 'salt': 'orange', 'saturates...",0005fc89f7,"[{'text': 'crustaceans, shrimp, raw (not previ...","[{'text': 'In a large bowl, toss the shrimp, g...","[{'fat': 4.6266384, 'nrg': 771.1064, 'pro': 18...","{'energy': 194.7525956849092, 'fat': 15.980767...",train,"[{'text': '2'}, {'text': '3/4'}, {'text': '1 1...",Shrimp and Caper Salad,"[{'text': 'pound'}, {'text': 'cup'}, {'text': ...",http://allrecipes.com/recipe/shrimp-and-caper-...,"[907.184, 75.0, 151.5, 25.799999999999997, 134..."
4,"{'fat': 'orange', 'salt': 'green', 'saturates'...",0006ca31f4,"[{'text': 'cocoa, dry powder, unsweetened'}, {...",[{'text': 'Measure out the cocoa powder into a...,"[{'fat': 141.35999999999999, 'nrg': 2352.0, 'p...","{'energy': 303.43539955190437, 'fat': 5.094846...",train,"[{'text': '12'}, {'text': '12'}, {'text': '1'}]",Natural Peanut Butter Chocolate Bon Bons,"[{'text': 'cup'}, {'text': 'cup'}, {'text': 'c...",http://www.food.com/recipe/natural-peanut-butt...,"[1032.0, 4068.0, 256.0]"


In [5]:
## read file
flavor_df = pd.read_json(flavor_path);
flavor_df.info()
flavor_df.iloc[:5, :]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934 entries, 0 to 933
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   category               934 non-null    object
 1   entity_id              934 non-null    int64 
 2   category_readable      934 non-null    object
 3   entity_alias_basket    934 non-null    object
 4   entity_alias_readable  934 non-null    object
 5   natural_source_name    934 non-null    object
 6   entity_alias           934 non-null    object
 7   molecules              934 non-null    object
 8   natural_source_url     934 non-null    object
 9   entity_alias_url       934 non-null    object
 10  entity_alias_synonyms  934 non-null    object
dtypes: int64(1), object(10)
memory usage: 80.4+ KB


,category,entity_id,category_readable,entity_alias_basket,entity_alias_readable,natural_source_name,entity_alias,molecules,natural_source_url,entity_alias_url,entity_alias_synonyms
0,animalproduct,0,Animal Product,"egg, egg-boiled, egg-cooked, egg-scrambled",Egg,Chicken,egg,"[{'bond_stereo_count': 0, 'undefined_atom_ster...",https://en.wikipedia.org/wiki/Chicken,https://en.wikipedia.org/wiki/Egg_as_food,Egg
1,bakery,1,Bakery,bakery-products,Bakery Products,Poacceae,bakery,"[{'bond_stereo_count': 0, 'undefined_atom_ster...",https://en.wikipedia.org/wiki/Poaceae,https://en.wikipedia.org/wiki/Bakery,Bakery Products
2,bakery,2,Bakery,"bread, bread-preferment",Bread,Poacceae,bread,"[{'bond_stereo_count': 0, 'undefined_atom_ster...",https://en.wikipedia.org/wiki/Poaceae,https://en.wikipedia.org/wiki/Bread,Bread
3,bakery,3,Bakery,bread-rye,Rye Bread,Rye,bread-rye,"[{'bond_stereo_count': 0, 'undefined_atom_ster...",https://en.wikipedia.org/wiki/Rye,https://en.wikipedia.org/wiki/Rye_bread,Rye Bread
4,bakery,4,Bakery,bread-wheaten,Wheaten Bread,Wheat,bread-wheaten,"[{'bond_stereo_count': 0, 'undefined_atom_ster...",https://en.wikipedia.org/wiki/Wheat,https://en.wikipedia.org/wiki/Soda_bread,"Soda scones, Soda farls"


In [36]:
## randomly pick 1000 entries
dev_df = food_df.sample(n=1000)

In [37]:
dev_df.info()
dev_df.iloc[:10,:]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 9920 to 10745
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   fsa_lights_per100g   1000 non-null   object
 1   id                   1000 non-null   object
 2   ingredients          1000 non-null   object
 3   instructions         1000 non-null   object
 4   nutr_per_ingredient  1000 non-null   object
 5   nutr_values_per100g  1000 non-null   object
 6   partition            1000 non-null   object
 7   quantity             1000 non-null   object
 8   title                1000 non-null   object
 9   unit                 1000 non-null   object
 10  url                  1000 non-null   object
 11  weight_per_ingr      1000 non-null   object
dtypes: object(12)
memory usage: 101.6+ KB


,fsa_lights_per100g,id,ingredients,instructions,nutr_per_ingredient,nutr_values_per100g,partition,quantity,title,unit,url,weight_per_ingr
9920,"{'fat': 'red', 'salt': 'orange', 'saturates': ...",316d1a0bd3,"[{'text': 'wheat flour, white, all-purpose, un...",[{'text': 'Here is one from The Tassajara Brea...,"[{'fat': 2.46, 'nrg': 910.0, 'pro': 25.82, 'sa...","{'energy': 489.07732106927284, 'fat': 31.20293...",test,"[{'text': '2'}, {'text': '1'}, {'text': '1'}, ...",Buckwheat Muffins Recipe,"[{'text': 'cup'}, {'text': 'tablespoon'}, {'te...",http://cookeatshare.com/recipes/buckwheat-muff...,"[250.0, 7.800000000000001, 6.0, 432.0]"
4191,"{'fat': 'orange', 'salt': 'orange', 'saturates...",14ea50e82f,"[{'text': 'wheat flours, bread, unenriched'}, ...",[{'text': 'Measure yeast directly into bread m...,"[{'fat': 0.165520833333333, 'nrg': 36.09374999...","{'energy': 266.6771489902788, 'fat': 4.2690356...",test,"[{'text': '3 1/2'}, {'text': '1'}, {'text': '2...",Fluffy White Bread,"[{'text': 'teaspoon'}, {'text': 'cup'}, {'text...",http://tastykitchen.com/recipes/breads/fluffy-...,"[9.989583333333332, 244.0, 42.375, 12.60000000..."
42881,"{'fat': 'orange', 'salt': 'orange', 'saturates...",d681c12f5b,"[{'text': 'water, bottled, generic'}, {'text':...",[{'text': 'Put all of the ingredients into the...,"[{'fat': 0.0, 'nrg': 0.0, 'pro': 0.0, 'sat': 0...","{'energy': 279.51964117774725, 'fat': 5.884974...",train,"[{'text': '1'}, {'text': '2'}, {'text': '1/4'}...",Wheat Grain Bread for Bread Machine,"[{'text': 'cup'}, {'text': 'tablespoon'}, {'te...",http://tastykitchen.com/recipes/breads/wheat-g...,"[236.8, 27.0, 84.75, 137.0, 41.2, 7.5, 7.1875,..."
30293,"{'fat': 'red', 'salt': 'green', 'saturates': '...",971ba5dceb,"[{'text': 'sugars, granulated'}, {'text': 'but...","[{'text': 'Line an 8"" Square pan with foil, le...","[{'fat': 0.0, 'nrg': 2304.0, 'pro': 0.0, 'sat'...","{'energy': 426.091614354174, 'fat': 43.8284913...",train,"[{'text': '3'}, {'text': '34'}, {'text': '23'}...",Cinnamon Fudge,"[{'text': 'cup'}, {'text': 'cup'}, {'text': 'c...",http://www.food.com/recipe/cinnamon-fudge-345272,"[604.8000000000001, 7724.799999999999, 5612.0,..."
46118,"{'fat': 'orange', 'salt': 'green', 'saturates'...",e654a4bd32,"[{'text': 'water, bottled, generic'}, {'text':...",[{'text': 'In medium saucepan over medium heat...,"[{'fat': 0.0, 'nrg': 0.0, 'pro': 0.0, 'sat': 0...","{'energy': 290.9659177238448, 'fat': 5.9237336...",train,"[{'text': '3'}, {'text': '12'}, {'text': '14'}...",Chocolate Walnut Gelato,"[{'text': 'cup'}, {'text': 'cup'}, {'text': 'c...",http://www.food.com/recipe/chocolate-walnut-ge...,"[710.4000000000001, 2419.2000000000003, 1204.0..."
41688,"{'fat': 'green', 'salt': 'orange', 'saturates'...",d05d169f8c,"[{'text': 'wheat flour, white, all-purpose, un...",[{'text': 'Mix 2 cups water with 1 cup ice cub...,"[{'fat': 5.535, 'nrg': 2047.5, 'pro': 58.095, ...","{'energy': 95.16206611455125, 'fat': 1.4864793...",train,"[{'text': '4 1/2'}, {'text': '2'}, {'text': '1...",Easy Buttermilk Pot Bread,"[{'text': 'cup'}, {'text': 'tablespoon'}, {'te...",http://www.vegetariantimes.com/recipe/easy-but...,"[562.5, 25.200000000000003, 10.499999999999998..."
42768,"{'fat': 'green', 'salt': 'green', 'saturates':...",d5ebb68f0a,"[{'text': 'kumquats, raw'}, {'text': 'water, b...","[{'text': 'Wash cumquats well.'}, {'text': 'Pu...","[{'fat': 8.6, 'nrg': 710.0, 'pro': 18.79999999...","{'energy': 91.1291882995182, 'fat': 0.24374365...",train,"[{'text': '1'}, {'text': '1 1/4'}, {'text': '1...",Cumquat Marmaladequat Marmalade,"[{'text': 'kg'}, {'text': 'liter'}, {'text': '...",http://www.food.com/recipe/cumquat-marmaladequ...,"[1000.0, 1251.1032000000002, 3416.0, 1250.0]"
50353,"{'fat': 'red', 'salt': 'green', 'saturates': '...",fb68061211,"[{'text': 'nuts, pecans'}, {'text': 'butter, w...",[{'text': 'Heat the oven to 150C/130C fan/gas ...,"[{'fat': 79.167, 'nrg': 760.1, 'pro': 10.087, ...","{'energy': 547.179378188001, 'fat': 41.2577119...",val,"[{'text': '1

In [6]:
## flatten molecules from nested array
flatten_flavor_df = flatten(flavor_path, ["molecules"], ["entity_id", "category", "entity_alias_readable"])
flatten_flavor_df.iloc[0:3,:]

,bond_stereo_count,undefined_atom_stereocenter_count,taste,functional_groups,fooddb_flavor_profile,supersweetdb_id,fema_number,fooddb_id,common_name,hba_count,...,hbd_count,fema_flavor_profile,complexity,heavy_atom_count,defined_atom_stereocenter_count,monoisotopic_mass,atom_stereo_count,entity_id,category,entity_alias_readable
0,0,0,,amine@primary amine@primary aliphatic amine (a...,faint,,3819,FDB002257,L-arginine,4,...,4,,176.0,12,1,174.112,1,0,animalproduct,Egg
1,0,0,"warm, over-ripe, fruity flavor below 0.1 ppm",aromatic compound@heterocyclic compound,very strong@mothball@animal@fecal@indole@civet,,3019,FDB010993,3-Methylindole,0,...,1,,122.0,10,0,131.073,0,0,animalproduct,Egg
2,0,0,,aromatic compound@heterocyclic compound,roast beef@roasted nuts@grass@medical@cocoa@wo...,,3272,FDB013954,"2,5-Dimethylpyrazine",2,...,0,"Cocoa, Roast Beef, Roasted Nut",62.9,8,0,108.069,0,0,animalproduct,Egg


In [7]:
## flatten ingredients from nested array
partition = 200
flatten_food_df = flatten(food_path, ["ingredients"], ["id", "title"])
flatten_food_df = flatten_food_df.iloc[0:partition, :] # partition to a small subset for testing

## re-order the column header and rename ingredient
flatten_food_df = flatten_food_df[["id", "title", "text"]]
flatten_food_df = flatten_food_df.rename(columns = {"text": "ingredient"})
flatten_food_df = flatten_food_df.drop_duplicates(subset=['id', 'ingredient']) # remove duplicated ingredients for each food
flatten_food_df = flatten_food_df.reset_index(drop=True)

In [8]:
flatten_food_df.info()
flatten_food_df.iloc[0:12, :]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          195 non-null    object
 1   title       195 non-null    object
 2   ingredient  195 non-null    object
dtypes: object(3)
memory usage: 4.7+ KB


,id,title,ingredient
0,000095fc1d,Yogurt Parfaits,"yogurt, greek, plain, nonfat"
1,000095fc1d,Yogurt Parfaits,"strawberries, raw"
2,000095fc1d,Yogurt Parfaits,"cereals ready-to-eat, granola, homemade"
3,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","sugars, granulated"
4,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","oil, corn, peanut, and olive"
5,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","egg substitute, powder"
6,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","orange juice, raw"
7,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","leavening agents, baking powder, double-acting..."
8,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","wheat flour, white, all-purpose, unenriched"
9,00059b093b,Honey Sriracha Chicken Wings,"chicken, broilers or fryers, wing, meat and sk..."


In [9]:
## count number of uninque ingredient for each food
## after deduplicated the ingredients, group them into list
unique_ingredient = flatten_food_df[["id", "ingredient"]].groupby(["id"]).agg(unique_ingredient=("ingredient", lambda x : list(x))).reset_index()
unique_ingredient["ingredient_count"] = unique_ingredient["unique_ingredient"].apply(lambda x : len(x))

In [10]:
unique_ingredient.info()
unique_ingredient.iloc[:10, :]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 32 non-null     object
 1   unique_ingredient  32 non-null     object
 2   ingredient_count   32 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 896.0+ bytes


,id,unique_ingredient,ingredient_count
0,000095fc1d,"[yogurt, greek, plain, nonfat, strawberries, r...",3
1,00051d5b9d,"[sugars, granulated, oil, corn, peanut, and ol...",6
2,00059b093b,"[chicken, broilers or fryers, wing, meat and s...",11
3,0005fc89f7,"[crustaceans, shrimp, raw (not previously froz...",8
4,0006ca31f4,"[cocoa, dry powder, unsweetened, honey, peanut...",3
5,00073a6b36,"[cheese, pasteurized process, american, withou...",4
6,0007c8edef,"[spartan, real semi-sweet chocolate baking chi...",5
7,0007f5f800,"[cornmeal, degermed, unenriched, yellow, wheat...",8
8,0008c6e39a,"[sugars, powdered, wheat flour, white, cake, e...",7
9,0009785a2e,"[rice, white, long-grain, regular, unenriched,...",8


In [11]:
## flatten, extract the molecule info
flatten_flavor_df = flatten_flavor_df[["category", "entity_alias_readable", "fooddb_id", "pubchem_id", "common_name", "iupac_name", "taste", "odor", "flavor_profile"]]
flatten_flavor_df.iloc[0:10, :]

,category,entity_alias_readable,fooddb_id,pubchem_id,common_name,iupac_name,taste,odor,flavor_profile
0,animalproduct,Egg,FDB002257,6322,L-arginine,(2S)-2-amino-5-(diaminomethylideneamino)pentan...,,,faint
1,animalproduct,Egg,FDB010993,6736,3-Methylindole,3-methyl-1H-indole,"warm, over-ripe, fruity flavor below 0.1 ppm","characteristic fecal odor at high levels, beco...",very strong@mothball@animal@fecal@indole@civet
2,animalproduct,Egg,FDB013954,31252,"2,5-Dimethylpyrazine","2,5-dimethylpyrazine",,,roast beef@roasted nuts@grass@medical@cocoa@wo...
3,animalproduct,Egg,FDB008174,7909,4-Methyl-2-pentanone,4-methylpentan-2-one,,"pleasant odor@faint, ketonic and camphor odor",solvent@spice@herbal@sharp@fruity@dairy@green
4,animalproduct,Egg,FDB008127,7284,2-Methylbutyraldehyde,2-methylbutanal,,,musty@coffee@cocoa@nutty@almond
5,animalproduct,Egg,FDB012552,7501,Styrene,styrene,,"extremely penetrating@an aromatic odor@sweet, ...",balsamic@gasoline@floral@sweet@balsam@plastic
6,animalproduct,Egg,FDB008326,9609,Diethyl sulfide,ethylsulfanylethane,,garlic-like odor@ethereal odor,ethereal@sulfurous
7,animalproduct,Egg,FDB012084,12180,Methyl butyrate,methyl butanoate,"apple-like, sweet taste@not very powerful@belo...",apple-like odor,apple@pineapple@ether@banana@fruit@sweet@fruity
8,animalproduct,Egg,FDB003349,18827,1-Octen-3-Ol,oct-1-en-3-ol,,,raw@fishy@oily@earthy@fungal@chicken@mushroom@...
9,animalproduct,Egg,FDB008541,18635,3-(Methylthio)propionaldehyde,3-methylsulfanylpropanal,,,potato@vegetable@tomato@fishy@musty@earthy@cre...


In [12]:
# ## fuzzywuzzy implementation
# ingredients = flavor_df["entity_alias_readable"].unique()
# start = time.process_time()
# food_df['key']=food_df['ingredient'].apply(lambda x : [process.extract(x, ingredients, limit=1)][0][0][0])
# duration = time.process_time() - start 
# print(duration)    

In [12]:
## levenshtein implementation
ingredients = flatten_flavor_df["entity_alias_readable"].unique()
start = time.process_time()
flatten_food_df['key']=flatten_food_df['ingredient'].apply(lambda x : min(ingredients, key=lambda i : lev(x, i))) 
duration = time.process_time() - start 
print(duration)  

0.1916240000000009


In [13]:
flatten_food_df.iloc[0:10, :]

,id,title,ingredient,key
0,000095fc1d,Yogurt Parfaits,"yogurt, greek, plain, nonfat",Northern bluefin tuna
1,000095fc1d,Yogurt Parfaits,"strawberries, raw",Strawberry Jam
2,000095fc1d,Yogurt Parfaits,"cereals ready-to-eat, granola, homemade",Wholewheat Bread
3,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","sugars, granulated",Pomegranate
4,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","oil, corn, peanut, and olive",European anchovy
5,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","egg substitute, powder",Sugar substitute
6,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","orange juice, raw",Orange Oil
7,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","leavening agents, baking powder, double-acting...",Mandarin Orange Peel Oil
8,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","wheat flour, white, all-purpose, unenriched",Breadnut tree seed
9,00059b093b,Honey Sriracha Chicken Wings,"chicken, broilers or fryers, wing, meat and sk...",New Zealand spinach


In [14]:
## join two dataset on ingredients
merged_df = flatten_flavor_df.merge(flatten_food_df,left_on='entity_alias_readable',right_on='key', how="inner")
merged_df = merged_df.sort_values(by=["id"])
merged_df = merged_df.reset_index(drop=True)
## reorder the cols
merged_df = merged_df[["id", "title", "ingredient", "category", "entity_alias_readable", "fooddb_id", "pubchem_id", "common_name", "iupac_name", "taste", "odor", "flavor_profile"]]
merged_df = merged_df.rename(columns = {"title": "food", "entity_alias_readable": "ingredient_flavorDB"})


In [17]:
## flatten col "nutrition per 100g"
## then join
# with open(food_path,'r') as f:
#     data = json.loads(f.read())
# nutr = pd.json_normalize(data)
# flatten_nutr = nutr.filter(regex="nutr_values_per100g")
# flatten_nutr.insert(0, "title", nutr["title"], True)
# flatten_nutr.insert(0, "id", nutr["id"], True)

In [15]:
merged_df.info()
merged_df.iloc[0:10, :]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9340 entries, 0 to 9339
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   9340 non-null   object
 1   food                 9340 non-null   object
 2   ingredient           9340 non-null   object
 3   category             9340 non-null   object
 4   ingredient_flavorDB  9340 non-null   object
 5   fooddb_id            9340 non-null   object
 6   pubchem_id           9340 non-null   int64 
 7   common_name          9340 non-null   object
 8   iupac_name           9340 non-null   object
 9   taste                9340 non-null   object
 10  odor                 9340 non-null   object
 11  flavor_profile       9340 non-null   object
dtypes: int64(1), object(11)
memory usage: 875.8+ KB


,id,food,ingredient,category,ingredient_flavorDB,fooddb_id,pubchem_id,common_name,iupac_name,taste,odor,flavor_profile
0,000095fc1d,Yogurt Parfaits,"strawberries, raw",fruit-berry,Strawberry Jam,FDB014891,445070,farnesol,"(2E,6E)-3,7,11-trimethyldodeca-2,6,10-trien-1-ol",weak citrus-lime taste,"delicate flowery odor@mild, oily@weak citrus-l...",orange@linden@mild@clove@fresh@waxy@floral@mug...
1,000095fc1d,Yogurt Parfaits,"cereals ready-to-eat, granola, homemade",bakery,Wholewheat Bread,,61209,"3-Methylcyclopentane-1,2-dione","3-methylcyclopentane-1,2-dione",,,
2,000095fc1d,Yogurt Parfaits,"cereals ready-to-eat, granola, homemade",bakery,Wholewheat Bread,FDB003296,11579,Levulinic Acid,4-oxopentanoic acid,,,caramel@acetoin@acidic@sweet@buttery
3,000095fc1d,Yogurt Parfaits,"cereals ready-to-eat, granola, homemade",bakery,Wholewheat Bread,FDB012137,8194,Dodecanal,dodecanal,,,citrus@fat@waxy@floral@lily@aldehydic@soapy@green
4,000095fc1d,Yogurt Parfaits,"cereals ready-to-eat, granola, homemade",bakery,Wholewheat Bread,FDB003311,5283335,trans-2-Nonenal,(E)-non-2-enal,,,waxy@green@fatty@paper@melon@cucumber
5,000095fc1d,Yogurt Parfaits,"cereals ready-to-eat, granola, homemade",bakery,Wholewheat Bread,FDB017700,72277,(-)-Epigallocatechin,"(2R,3R)-2-(3,4,5-trihydroxyphenyl)-3,4-dihydro...",,,bitter
6,000095fc1d,Yogurt Parfaits,"cereals ready-to-eat, granola, homemade",bakery,Wholewheat Bread,FDB002022,15037,1-Furfurylpyrrole,1-(furan-2-ylmethyl)pyrrole,,,vegetable@waxy@plastic@coffee@fruity@green
7,000095fc1d,Yogurt Parfaits,"cereals ready-to-eat, granola, homemade",bakery,Wholewheat Bread,FDB011886,878,methanethiol,methanethiol,unpleasant cabbage taste,"odor of rotten cabbage@sulfidy@powerful, unple...",sulfur@garlic@gasoline@decomposing cabbage
8,000095fc1d,Yogurt Parfaits,"cereals ready-to-eat, granola, homemade",bakery,Wholewheat Bread,FDB012549,7997,Propyl acetate,propyl acetate,"pleasant, bittersweet flavor reminiscent of pe...","pleasant odor@odor of pears@mild, fruity odor",solvent@fusel@pear@celery@raspberry@bitter@fruity
9,000095fc1d,Yogurt Parfaits,"cereals ready-to-eat, granola, homemade",bakery,Wholewheat Bread,FDB000736,10931,"3,3-Dimethylacrylic acid",3-methylbut-2-enoic acid,,,alcohol@dairy@phenolic@green


In [16]:
## count the freq of category of ingredients for each food
category_freq = merged_df[["id", "food", "ingredient", "category"]].drop_duplicates(subset=["id", "ingredient"]).groupby(['id', 'food','category'],  as_index=False)['id'].agg({'Frequency':'count'})


In [17]:
category_freq.info()
category_freq.iloc[:10, :]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         156 non-null    object
 1   food       156 non-null    object
 2   category   156 non-null    object
 3   Frequency  156 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 5.0+ KB


,id,food,category,Frequency
0,000095fc1d,Yogurt Parfaits,bakery,1
1,000095fc1d,Yogurt Parfaits,fishseafood-fish,1
2,000095fc1d,Yogurt Parfaits,fruit-berry,1
3,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe",additive,1
4,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe",essentialoil,2
5,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe",fishseafood-fish,1
6,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe",fruit,1
7,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe",plantderivative,1
8,00059b093b,Honey Sriracha Chicken Wings,additive,1
9,00059b093b,Honey Sriracha Chicken Wings,dish,1


In [18]:
## group the category freq into {key, val} pairs
category_freq["count"] = category_freq.apply(lambda x : {x["category"]: x["Frequency"]}, axis=1)
category_freq = category_freq.groupby(["id"], as_index=False).agg(category_freq=("count", lambda x : list(x)))

In [19]:
category_freq.iloc[:5, :]

,id,category_freq
0,000095fc1d,"[{'bakery': 1}, {'fishseafood-fish': 1}, {'fru..."
1,00051d5b9d,"[{'additive': 1}, {'essentialoil': 2}, {'fishs..."
2,00059b093b,"[{'additive': 1}, {'dish': 1}, {'essentialoil'..."
3,0005fc89f7,"[{'dish': 2}, {'essentialoil': 4}, {'fishseafo..."
4,0006ca31f4,[{'plantderivative': 3}]


In [20]:
## make set of molecules for each food
unique_molecules = merged_df[["id", "food", "ingredient_flavorDB", "fooddb_id", "pubchem_id", "common_name", "taste", "odor", "flavor_profile"]]
unique_molecules = unique_molecules.drop_duplicates(subset=["id", "pubchem_id"])

In [21]:
display(unique_molecules.iloc[:10, :])
unique_molecules.info()

,id,food,ingredient_flavorDB,fooddb_id,pubchem_id,common_name,taste,odor,flavor_profile
0,000095fc1d,Yogurt Parfaits,Strawberry Jam,FDB014891,445070,farnesol,weak citrus-lime taste,"delicate flowery odor@mild, oily@weak citrus-l...",orange@linden@mild@clove@fresh@waxy@floral@mug...
1,000095fc1d,Yogurt Parfaits,Wholewheat Bread,,61209,"3-Methylcyclopentane-1,2-dione",,,
2,000095fc1d,Yogurt Parfaits,Wholewheat Bread,FDB003296,11579,Levulinic Acid,,,caramel@acetoin@acidic@sweet@buttery
3,000095fc1d,Yogurt Parfaits,Wholewheat Bread,FDB012137,8194,Dodecanal,,,citrus@fat@waxy@floral@lily@aldehydic@soapy@green
4,000095fc1d,Yogurt Parfaits,Wholewheat Bread,FDB003311,5283335,trans-2-Nonenal,,,waxy@green@fatty@paper@melon@cucumber
5,000095fc1d,Yogurt Parfaits,Wholewheat Bread,FDB017700,72277,(-)-Epigallocatechin,,,bitter
6,000095fc1d,Yogurt Parfaits,Wholewheat Bread,FDB002022,15037,1-Furfurylpyrrole,,,vegetable@waxy@plastic@coffee@fruity@green
7,000095fc1d,Yogurt Parfaits,Wholewheat Bread,FDB011886,878,methanethiol,unpleasant cabbage taste,"odor of rotten cabbage@sulfidy@powerful, unple...",sulfur@garlic@gasoline@decomposing cabbage
8,000095fc1d,Yogurt Parfaits,Wholewheat Bread,FDB012549,7997,Propyl acetate,"pleasant, bittersweet flavor reminiscent of pe...","pleasant odor@odor of pears@mild, fruity odor",solvent@fusel@pear@celery@raspberry@bitter@fruity
9,000095fc1d,Yogurt Parfaits,Wholewheat Bread,FDB000736,10931,"3,3-Dimethylacrylic acid",,,alcohol@dairy@phenolic@green


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5424 entries, 0 to 9339
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   5424 non-null   object
 1   food                 5424 non-null   object
 2   ingredient_flavorDB  5424 non-null   object
 3   fooddb_id            5424 non-null   object
 4   pubchem_id           5424 non-null   int64 
 5   common_name          5424 non-null   object
 6   taste                5424 non-null   object
 7   odor                 5424 non-null   object
 8   flavor_profile       5424 non-null   object
dtypes: int64(1), object(8)
memory usage: 423.8+ KB


In [22]:
## make molecule attributes into {key: val} pair
columns = ["ingredient_flavorDB", "fooddb_id", "pubchem_id", "common_name", "taste", "odor", "flavor_profile"]
unique_molecules["molecules"] = unique_molecules[columns].to_dict(orient="records")

## group by each food
unique_molecules = unique_molecules[["id", "food", "molecules"]]
unique_molecules = unique_molecules.groupby(["id", "food"], as_index=False).agg(flavor_molecules=("molecules", lambda x : list(x)))


In [29]:
display(unique_molecules)

,id,food,flavor_molecules
0,000095fc1d,Yogurt Parfaits,"[{'ingredient_flavorDB': 'Strawberry Jam', 'fo..."
1,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","[{'ingredient_flavorDB': 'Pomegranate', 'foodd..."
2,00059b093b,Honey Sriracha Chicken Wings,"[{'ingredient_flavorDB': 'Black huckleberry', ..."
3,0005fc89f7,Shrimp and Caper Salad,[{'ingredient_flavorDB': 'Mandarin Orange Peel...
4,0006ca31f4,Natural Peanut Butter Chocolate Bon Bons,"[{'ingredient_flavorDB': 'Honey', 'fooddb_id':..."
5,00073a6b36,Easy Cheese Sauce,[{'ingredient_flavorDB': 'Spineless Monkey Ora...
6,0007c8edef,Easy Chocolate Frosting Recipe,"[{'ingredient_flavorDB': 'Japanese chestnut', ..."
7,0007f5f800,Cornmeal Crackers with Pumpkin Seeds,[{'ingredient_flavorDB': 'Spineless Monkey Ora...
8,0008c6e39a,Chocolate-Orange Angel Food Cake,"[{'ingredient_flavorDB': 'Pomegranate', 'foodd..."
9,0009785a2e,"Rice With Lemon, Capers & Parsley","[{'ingredient_flavorDB': 'Butternut squash', '..."


In [30]:
## count number of distinct flavor_molecules for each food
unique_molecules["molecules_count"] = unique_molecules.apply(lambda x : len(x['flavor_molecules']), axis=1)

In [31]:
display(unique_molecules.iloc[0:5, :])
unique_molecules.info()

,id,food,flavor_molecules,molecules_count
0,000095fc1d,Yogurt Parfaits,"[{'ingredient_flavorDB': 'Strawberry Jam', 'fo...",83
1,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","[{'ingredient_flavorDB': 'Pomegranate', 'foodd...",172
2,00059b093b,Honey Sriracha Chicken Wings,"[{'ingredient_flavorDB': 'Black huckleberry', ...",267
3,0005fc89f7,Shrimp and Caper Salad,[{'ingredient_flavorDB': 'Mandarin Orange Peel...,259
4,0006ca31f4,Natural Peanut Butter Chocolate Bon Bons,"[{'ingredient_flavorDB': 'Honey', 'fooddb_id':...",75


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                32 non-null     object
 1   food              32 non-null     object
 2   flavor_molecules  32 non-null     object
 3   molecules_count   32 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 1.1+ KB


In [32]:
# add attributes: fsa_lights_per100g, ingredients, nutr_values_per100g
final_df = unique_molecules.merge(food_df[["id","instructions", "nutr_values_per100g", "fsa_lights_per100g"]], on="id", how="inner")

## add unique ingredients for each food
final_df = final_df.merge(unique_ingredient, on="id", how="left")
# add category freq
final_df = final_df.merge(category_freq, on="id", how="left")

# reorder column
final_df = final_df[["id", "food",  "unique_ingredient",  "ingredient_count", "category_freq", "instructions", "nutr_values_per100g", "fsa_lights_per100g", "flavor_molecules", "molecules_count"]]

In [33]:
final_df.info()
display(final_df.iloc[:10, :])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 0 to 31
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   32 non-null     object
 1   food                 32 non-null     object
 2   unique_ingredient    32 non-null     object
 3   ingredient_count     32 non-null     int64 
 4   category_freq        32 non-null     object
 5   instructions         32 non-null     object
 6   nutr_values_per100g  32 non-null     object
 7   fsa_lights_per100g   32 non-null     object
 8   flavor_molecules     32 non-null     object
 9   molecules_count      32 non-null     int64 
dtypes: int64(2), object(8)
memory usage: 2.8+ KB


,id,food,unique_ingredient,ingredient_count,category_freq,instructions,nutr_values_per100g,fsa_lights_per100g,flavor_molecules,molecules_count
0,000095fc1d,Yogurt Parfaits,"[yogurt, greek, plain, nonfat, strawberries, r...",3,"[{'bakery': 1}, {'fishseafood-fish': 1}, {'fru...",[{'text': 'Layer all ingredients in a serving ...,"{'energy': 81.12946131894766, 'fat': 2.1401392...","{'fat': 'green', 'salt': 'green', 'saturates':...","[{'ingredient_flavorDB': 'Strawberry Jam', 'fo...",83
1,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","[sugars, granulated, oil, corn, peanut, and ol...",6,"[{'additive': 1}, {'essentialoil': 2}, {'fishs...",[{'text': 'Cream sugar and butter together til...,"{'energy': 477.09640393594606, 'fat': 23.41248...","{'fat': 'red', 'salt': 'orange', 'saturates': ...","[{'ingredient_flavorDB': 'Pomegranate', 'foodd...",172
2,00059b093b,Honey Sriracha Chicken Wings,"[chicken, broilers or fryers, wing, meat and s...",11,"[{'additive': 1}, {'dish': 1}, {'essentialoil'...","[{'text': 'Preheat oven to 400 degrees F.'}, {...","{'energy': 208.05898280960727, 'fat': 14.29704...","{'fat': 'orange', 'salt': 'orange', 'saturates...","[{'ingredient_flavorDB': 'Black huckleberry', ...",267
3,0005fc89f7,Shrimp and Caper Salad,"[crustaceans, shrimp, raw (not previously froz...",8,"[{'dish': 2}, {'essentialoil': 4}, {'fishseafo...","[{'text': 'In a large bowl, toss the shrimp, g...","{'energy': 194.7525956849092, 'fat': 15.980767...","{'fat': 'orange', 'salt': 'orange', 'saturates...",[{'ingredient_flavorDB': 'Mandarin Orange Peel...,259
4,0006ca31f4,Natural Peanut Butter Chocolate Bon Bons,"[cocoa, dry powder, unsweetened, honey, peanut...",3,[{'plantderivative': 3}],[{'text': 'Measure out the cocoa powder into a...,"{'energy': 303.43539955190437, 'fat': 5.094846...","{'fat': 'orange', 'salt': 'green', 'saturates'...","[{'ingredient_flavorDB': 'Honey', 'fooddb_id':...",75
5,00073a6b36,Easy Cheese Sauce,"[cheese, pasteurized process, american, withou...",4,"[{'essentialoil': 1}, {'fishseafood-fish': 1},...","[{'text': 'Put milk, onion salt and pepper sau...","{'energy': 57.123531501115025, 'fat': 2.477641...","{'fat': 'green', 'salt': 'red', 'saturates': '...",[{'ingredient_flavorDB': 'Spineless Monkey Ora...,33
6,0007c8edef,Easy Chocolate Frosting Recipe,"[spartan, real semi-sweet chocolate baking chi...",5,"[{'bakery': 1}, {'essentialoil': 1}, {'fruit':...",[{'text': 'Heat chocolate on top of double boi...,"{'energy': 367.7230503667077, 'fat': 4.3421179...","{'fat': 'orange', 'salt': 'orange', 'saturates...","[{'ingredient_flavorDB': 'Japanese chestnut', ...",168
7,0007f5f800,Cornmeal Crackers with Pumpkin Seeds,"[cornmeal, degermed, unenriched, yellow, wheat...",8,"[{'dairy': 1}, {'essentialoil': 1}, {'fishseaf...","[{'text': 'In a large skillet, stir the cornme...","{'energy': 399.5182190718795, 'fat': 18.337440...","{'fat': 'orange', 'salt': 'red', 'saturates': ...",[{'ingredient_flavorDB': 'Spineless Monkey Ora...,156
8,0008c6e39a,Chocolate-Orange Angel Food Cake,"[sugars, powdered, wheat flour, white, cake, e...",7,"[{'additive': 1}, {'essentialoil': 2}, {'fishs...",[{'text': 'Move oven rack to lowest position.'...,"{'energy': 149.42599870037444, 'fat': 0.367917...","{'fat': 'green', 'salt': 'green', 'saturates':...","[{'ingredient_flavorDB': 'Pomegranate', 'foodd...",164
9,0009785a2e,"Rice With Lemon, Capers & Parsley","[rice, white, long-grain, regular, unenriched,...",8,"[{'beverage-alcoholic': 1}, {'essentialoil': 4...",[{'text': 'Add rice to a large pan of boiling ...,"{'energy': 57.34966592427616, 'fat': 3.4322726...","{'fat': 'orange', 'salt': 'red', 'saturates': ...","[{'ingredient_flavorDB': 'Butternut squash', '...",281


In [34]:
## verify the freq of ingredient category, sum of freq should be equal to number of ingredients
final_df['category_freq'].apply(lambda x : sum(list(e.values())[0] for e in x))
final_df[final_df["ingredient_count"] != final_df['category_freq'].apply(lambda x : sum(list(e.values())[0] for e in x))].shape[0]

0

In [35]:
final_df.to_json(r'schema.json', orient="records")
print("writing finished")

writing finished
